In [1]:
import math
import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
from sklearn import metrics, datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

random_seed = 30

In [2]:
data = pd.read_csv("../data/featured/data.csv", sep=';')

Xdata = data.drop('teamA_win', axis=1)
ydata = data.teamA_win

In [3]:
Xtrain, Xval, ytrain, yval = train_test_split(Xdata, ydata, test_size=0.4, random_state=random_seed)

In [4]:
clfLOG = LogisticRegression(solver='newton-cg')
clfLOG.fit(Xtrain,ytrain)

print('\nValidační přesnost:', clfLOG.score(Xval,yval))


Validační přesnost: 0.7119155354449472
